In [ ]:
import numpy as np
import pandas as pd
import spacy

nlp = spacy.load('en_core_web_lg')

df = pd.DataFrame({'Questions': ['What are you doing?','What are you doing tonight?','What are you doing now?','What is your name?','What is your nick name?','What is your full name?','Shall we meet?',
                             'How are you doing?' ]})

def convert_to_doc(text):
    doc = nlp(text)
    return doc

def similarity_score(s1, s2):
    x = s1.similarity(s2)
    return 1.0 if x >= 0.9 else 0.0

def score_balancer(text1, text2):
    doc1 = convert_to_doc(text1)
    doc2 = convert_to_doc(text2)
    return (similarity_score(doc1, doc2) + similarity_score(doc2, doc1)) / 2

def column_similarity(x, df):
    sim_score = []
    for i in df['Questions']:
        sim_score.append(score_balancer(x, i))
    return sim_score

df['similarity'] = df['Questions'].apply(lambda x : column_similarity(x,df)).astype(str)

appended_data = []
for i, j in enumerate(df.groupby('similarity')['Questions']):
    data = pd.DataFrame(j[1])
    data['cluster'] = i
    appended_data.append(data)
appended_data_ = pd.concat(appended_data)

cluster_code = {0 : 'meeting', 1 : 'name', 2 : 'action'}
appended_data_ = appended_data_.replace({'cluster': cluster_code})
appended_data_